In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

/home/lzw365/anaconda3/envs/cse599/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
sentences = [
    "Small study shows a possible reason some long COVID patients experience 'brain fog'",
    "A minor study suggests a potential explanation for why some patients with long COVID suffer from cognitive impairments.",
    "I am smart",
    "I am stupid"
]

# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [14]:
outputs = model(**tokens)
type(outputs.keys())

odict_keys

In [15]:
embeddings = outputs.last_hidden_state

In [16]:
print(embeddings.shape)

torch.Size([4, 128, 768])


In [18]:
attention_mask = tokens['attention_mask']
print(attention_mask.shape)

torch.Size([4, 128])


In [19]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()
print(mask.shape)

torch.Size([4, 128, 768])


In [20]:
masked_embeddings = embeddings * mask

In [21]:
summed = torch.sum(masked_embeddings, 1)

In [22]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)

In [23]:
mean_pooled = summed / summed_mask

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.9023557, 0.2148076, 0.2281115]], dtype=float32)